This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Apply calibration solutions to dataset and check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with only gain corruptions and a gain-only solver,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-11-25 13:22:38,531 - pipeline.bandpass_calibration - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-11-25 13:22:38,544 - workflow.utils - INFO - Using AA2-Low-ECP-240228 with 68 stations
2024-11-25 13:22:39,733 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-11-25 13:22:39,734 - workflow.utils - INFO - Using 37 components from /data/EOS_1/mit183/gleamegc.dat


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-11-25 13:22:43,966 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-11-25 13:22:45,353 - pipeline.bandpass_calibration - INFO - Reading demo.ms in 16-channel chunks
2024-11-25 13:22:45,517 - pipeline.bandpass_calibration - INFO - Generating /data/EOS_1/mit183/gleamegc.dat LSM < 5.0 deg > 1 Jy
2024-11-25 13:22:46,183 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-11-25 13:22:46,184 - pipeline.bandpass_calibration - INFO - Predicting model visibilities in 16-channel chunks
2024-11-25 13:22:46,212 - pipeline.bandpass_calibration - INFO - Running calibration in 16-channel chunks
2024-11-25 13:22:46,265 - pipeline.bandpass_calibration - INFO - Applying solutions
2024-11-25 13:22:46,300 - pipeline.bandpass_calibration - INFO - Checking results
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<fr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-11-25 13:22:51,718 - pipeline.bandpass_calibration - INFO - Convergence checks passed
2024-11-25 13:22:51,732 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
/u/mit183/.pyenv/versions/3.11.7/lib/python3.11/site-packages/ska_sdp_datamodels/visibility/vis_model.py:190: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'baselines' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  return cls(datavars, coords=coords, attrs=attrs)
/u/mit183/.pyenv/versions/3.11.7/lib/python3.11/site-pac

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

In [4]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-11-25 13:22:55,121 - pipeline.bandpass_calibration - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-11-25 13:22:55,133 - workflow.utils - INFO - Using AA2-Low-ECP-240228 with 68 stations
2024-11-25 13:22:55,760 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-11-25 13:22:55,761 - workflow.utils - INFO - Using 37 components from /data/EOS_1/mit183/gleamegc.dat


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-11-25 13:22:59,950 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-11-25 13:22:59,952 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2024-11-25 13:23:01,413 - pipeline.bandpass_calibration - INFO - Reading demo.ms in 16-channel chunks
2024-11-25 13:23:01,521 - pipeline.bandpass_calibration - INFO - Generating /data/EOS_1/mit183/gleamegc.dat LSM < 5.0 deg > 1 Jy
2024-11-25 13:23:02,336 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-11-25 13:23:02,337 - pipeline.bandpass_calibration - INFO - Predicting model visibilities in 16-channel chunks
2024-11-25 13:23:02,374 - pipeline.bandpass_calibration - INFO - Running calibration in 16-channel chunks
2024-11-25 13:23:02,432 - pipeline.bandpass_calibration - INFO - Applying solutions
2024-11-25 13:23:02,515 - pipeline.bandpass_calibration - INFO - Checking results
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-11-25 13:23:16,727 - pipeline.bandpass_calibration - INFO - Convergence checks passed
2024-11-25 13:23:16,738 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
/u/mit183/.pyenv/versions/3.11.7/lib/python3.11/site-packages/ska_sdp_datamodels/visibility/vis_model.py:190: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'baselines' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  return cls(datavars, coords=coords, attrs=attrs)
/u/mit183/.pyenv/versions/3.11.7/lib/python3.11/site-pac

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 